## 2. Limpeza, Normalização e Padronização de dados

O pré-processamento de dados é uma etapa importante para o processo de análise de dados, pois a qualidade do resultado do seu modelo começa com a qualidade dos dados que você está “inputando”. Assim, parte considerável do tempo do cientista de dados é gasto no esforço que envolve a limpeza de dados e a engenharia de recursos (transformar dados brutos em atributos que melhor representem seus dados). Independentemente de o cientista de dados receber dados coletados ou ter que realizar a coleta, os dados estarão em formato bruto, que precisarão ser convertidos e filtrados.

### 2.1 Import

In [ ]:
import pandas as pd
from scipy import stats
import datetime
import numpy as np 
from sklearn import preprocessing


pd.set_option('display.max_columns', 30)


### 2.2 Carregando o dataset

O dataset da atividade anterior foi aumentado com novas colunas para simular caracteristicas de dados brutos.

In [ ]:
meu_data_frame = pd.read_pickle("../data/ugly_cereal.pkl")
display(meu_data_frame.head(20))

## 2.3 Limpeza de dados

### 2.3.1 Eliminando atributos redundantes

Neste exemplo, percebe-se que há redundancia entre as colunas "data_cre_scorp" e "data_cre_seman". Aparentemente o valor deste atributo em cada amostra é o mesmo, mas em formatos diferentes. Vamos checar.

In [ ]:

#flag que marca se as colunas são iguais
sao_iguais = True

#iterando sobre a coluna "data_cre_saman"
for i in range(len(meu_data_frame["data_cre_saman"].values)):
    #convertendo os valores de "data_cre_saman" para "data_cre_saman", ex: Sunday 30. April 1995 -> 1195-4-30
    alter_data = str(datetime.datetime.strptime(meu_data_frame["data_cre_saman"].values[i], "%A %d. %B %Y")).replace(" 00:00:00","")
    comp_data = str(meu_data_frame["data_cre_scorp"].values[i])
    
    #caso um único exemplo seja diferente, a flag é mercada como false
    if comp_data != alter_data:
            sao_iguais = False
            break

if sao_iguais:
    print("Há redundância de informação, vou deletar a coluna 'data_cre_saman'")
    meu_data_frame.drop(columns=['data_cre_saman'], axis=1, inplace=True) 
else:
    print("Não são iguais, é melhor pesquisar um pouco mais sobre a natureza desses atributos")
    
display(meu_data_frame.head(10))

Além das colunas redundantes, colunas com grande quantidade de dados nulos também devem ser removidas antes da filtragem por amostra.

### 2.3.2 Removendo amostras com valores de atributos nulos

In [ ]:
#um boa prática que antecede a remoção de valores nulos é a conversão de valores inválidos para nulos
#por exemplo, campos com espaço em branco, caracteres especiais sem significado (?,*,.), etc.
#Um regex pode ser usado para converter valores para NaN
meu_data_frame = meu_data_frame.replace(r'^\s*$', float("NaN"), regex=True)

#guardar as amostras irregulares é uma boa prática, lembre-se que esses dados podem ser revisados
#e podem ser úteis no futuro.
removed_data_frame = meu_data_frame[meu_data_frame.isnull().any(axis=1)].copy()

#deletando as amostram que possuem algum valor NaN em qualquer atributo 
meu_data_frame = meu_data_frame.dropna()

display(removed_data_frame)

### 2.3.3 Removendo amostras duplicadas

In [ ]:
print("Dimensões do dataset (linha,coluna):",  meu_data_frame.shape) 
meu_data_frame = meu_data_frame.drop_duplicates()
print("Dimensões do dataset (linha,coluna) após eliminar duplicatas:",  meu_data_frame.shape) 

### 2.3.4 Remoção de símbolos especiais, escalas de medidas e grandezas numericas

In [ ]:
#removendo o "un." dos valores da coluna "sales_week" e convertendo de "object" para "int64"
meu_data_frame["sales_week"] = meu_data_frame["sales_week"].str.replace("un.","")
meu_data_frame["sales_week"] = meu_data_frame["sales_week"].astype('int64')
display(meu_data_frame.head(10))

In [ ]:
#removendo o cifrão dos valores da coluna "price" e convertendo de "object" para "float64"
meu_data_frame["price"] = meu_data_frame["price"].str.replace("$","")
meu_data_frame["price"] = meu_data_frame["price"].astype('float64')
display(meu_data_frame.head(10))

### 2.3.5 Codificação de categorias

In [ ]:
#dados categoricos são convertidos para representação numérica (escala nominal)
meu_data_frame["mfr"] = meu_data_frame["mfr"].cat.codes
meu_data_frame["type"] = meu_data_frame["type"].cat.codes
display(meu_data_frame.head(10))

## 2.4 Normalização e padronização

A transformação dos seus dados, que já estão tratados, é uma pratica que tem vários impactos positivos na área de Ciência de Dados. Além de facilitar a visualizalção dos dados, a normalização e a padronização evitar que seu algoritmo  de aprendizado de máquina fique enviesado para as variáveis com maior ordem de grandeza.

* A normalização tem como objetivo converter a distribuição original para uma distribuição dentro de um intervalo, por exemplo: [0,1] ou [-1,1]

* A padronização tem como objetivo converter a distribuição original para uma distribuição com média 0 e desvio padrão 1.


In [ ]:
#instanciando o normalizador MinMAx
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

#obtendo as colunas do dataframe
columns = meu_data_frame.columns

#fazendo uma copia do dataframe, iremos normalizar a copia
normalized_data_frame = meu_data_frame.copy()

#iterando sobre cada coluna
for column in columns:
    #verificando se a coluna é numérica
    if(meu_data_frame[column].dtype == "int64" or meu_data_frame[column].dtype == "float64"):
        x = meu_data_frame[column].values
        x_norm = min_max_scaler.fit_transform(x.reshape(-1, 1))
        normalized_data_frame[column] = pd.DataFrame(x_norm)
        normalized_data_frame.rename(columns={column:column+"_norm"}, inplace=True)

display(normalized_data_frame.head(10))

In [ ]:

standard_scaler = preprocessing.StandardScaler()
columns = meu_data_frame.columns

standarlized_data_frame = meu_data_frame.copy()

for column in columns:
    if(meu_data_frame[column].dtype == "int64" or meu_data_frame[column].dtype == "float64"):
        x = meu_data_frame[column].values
        x_norm = standard_scaler.fit_transform(x.reshape(-1, 1))
        standarlized_data_frame[column] = pd.DataFrame(x_norm)
        standarlized_data_frame.rename(columns={column:column+"_stda"}, inplace=True)
display(standarlized_data_frame.head(10))

## 2.5 Correlação de atributos

O termo correlação representa, sob o ponto de vista da estatística, uma medida de associação
entre duas ou mais variáveis. Por definição, se forem considerados numa população, os pares de valores de duas variáveis (xi;yi), a correlação pode ser definida pela equação de Pearson abaixo:

<img src="imgs/corr.png" width=35% />

O valor da correção, conhecido como coeficiente de correlação, assume valores no intervalo de -1 a 1, de acordo com o grau de associação entre as variáveis em questão.

In [ ]:
#calculando a tabela de correlação
corr = meu_data_frame.corr()

p = 0.75 # correlação mínima
var = []

#iterando sobre a tabela
for i in corr.columns:
    for j in corr.columns:
        if(i != j):
            if np.abs(corr[i][j]) > p: # se maior do que |p|
                var.append([i,j])
print('Variáveis mais correlacionadas:\n', var)